In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn.datasets as datasets
import pandas as pd
import sklearn.metrics as mt
import pickle
import seaborn as sns


class NnModel:
    def __init__(self, x: np.ndarray, y: np.ndarray, hidden_neurons: int = 10, output_neurons: int = 2):
        np.random.seed(8)
        self.x = x
        self.y = y
        self.hidden_neurons = hidden_neurons
        self.output_neurons = output_neurons
        self.input_neurons = self.x.shape[1]

        # Inicializa os pesos e bias
        # Xavier Inicialization -> Variancia dos pesos igual em todas as camadas 
        # -> divide tudo por np.sqrt(self.input_neurons)

        self.W1 = np.random.randn(self.input_neurons, self.hidden_neurons) / np.sqrt(self.input_neurons)
        self.B1 = np.zeros((1, self.hidden_neurons))
        self.W2 = np.random.randn(self.hidden_neurons, self.output_neurons) / np.sqrt(self.hidden_neurons)
        self.B2 = np.zeros((1, self.output_neurons))
        self.model_dict = {"W1": self.W1, "B1": self.B1, "W2": self.W2, "B2": self.B2}
        self.z1 = 0
        self.f1 = 0


    def forward(self, x: np.ndarray) -> np.ndarray:
        # Equação da reta
        self.z1 = x.dot(self.W1) + self.B1

        # Função de ativação (1)

        self.f1 = np.tanh(self.z1)

        # Equação da reta (2)
        z2 = self.f1.dot(self.W2) + self.B2

     

        #Softmax
        exp_values = np.exp(z2)
        softmax = exp_values / np.sum(exp_values, axis = 1, keepdims = True) #soma da linha e não do array todo
        return softmax

        
        

    def loss(self, softmax):
        # Cross Entropy

        predictions = np.zeros(self.y.shape[0])

        for i, correct_index in enumerate(self.y):
            predicted = softmax[i][correct_index] #pega o valor do softmax e seu index
            predictions[i] = predicted

        log_prob = -np.log(predicted)
        return log_prob/self.y.shape[0]




    def backpropagation(self, softmax: np.ndarray, learning_rate: float) -> None:
        delta2 = np.copy(softmax)
        delta2[range(self.x.shape[0]), self.y] -= 1

        dW2 = (self.f1.T).dot(delta2)
        dB2 = np.sum(delta2, axis = 0, keepdims = True)

        delta1 = delta2.dot(self.W2.T)*(1-np.power(np.tanh(self.z1),2))
        dW1 = (self.x.T).dot(delta1)
        dB1 = np.sum(delta1)

        #atualização dos pesos e bias

        self.W1 += - learning_rate*dW1
        self.W2 += - learning_rate*dW2
        self.B1 += - learning_rate*dB1
        self.B2 += - learning_rate*dB2
        
    def show_plot(self, predictions):
        if self.x.shape[1] == 2:
            plt.scatter(self.x[:,0], self.x[:,1], s=50, c = predictions, cmap="cool", alpha=0.7)
            plt.show()
        elif self.x.shape[1] == 3:
            ax = plt.axes(projection = "3d")
            ax.scatter3D(x_[:,0],x_[:,1],x_[:,2], s=40, c=predictions, cmap="cool", alpha=0.8)
            plt.show()


            
        
    


    def fit(self, epochs: int, lr: float, show_plot: bool = False):

        for epoch in range(epochs):

            outputs = self.forward(self.x)
            loss = self.loss(outputs)
            
            self.backpropagation(outputs, lr)

            # Acuracia 
            prediction = np.argmax(outputs, axis = 1)
            correct = (prediction == self.y).sum()
            accuracy = correct/self.y.shape[0]

            if int((epoch+1) % (epochs/10)) == 0:
                print(f"Epoch: [{epoch + 1} / {epochs}] Accuracy: {accuracy:.3f} Loss: {loss.item():.5f}")
                if show_plot:
                    self.show_plot(prediction)
        return prediction


        